# Проект Bank Marketing

Автор: Олег Акимов (Slack: @Oleg)

## 0. Постановка задачи

В ходе проекта решается задача прогнозирования положительного отклика (срочный вклад) на прямые звонки из банка.  
По-сути, задача является задачей банковского скоринга, т.е. по характеристикам клиента (потенциального клиента), предсказывается его поведение (невозврат кредита, желание открыть вклад и т.д.). В департаментах рисков банков существуеют строгие требования к интерпретируемости модели, поэтому я хочу рассмотреть 2 решения: "банковское" и свободное. Задача обзвона клиентов не является задачей определения риска дефолта, поэтому на неё не накладывается условие интерпретируемости, однако задача схожая, и думаю интересно на ее примере показать решение с ограничением (интерпретируемость).

Источник данных UCI: https://archive.ics.uci.edu/ml/datasets/bank+marketing

## 1. Описание набора данных и признаков

Датасет содержит 41188 объектов, для кадого из которых задан 21 признак, в том числе 1 целевой. 

Рассмотрим по порядку все имеющиеся признаки:  
1) **age** - возраст потенциального клиента в годах  
2) **job** - профессия (Возможны следующие значения: *admin., blue-collar, entrepreneur, housemaid, management, retired, self-employed, services, student, technician, unemployed, unknown*)  
3) **marital** - семейное положение (Значения: *divorced, married, single, unknown*)  
4) **education** - образование (Значения: *basic.4y, basic.6y, basic.9y, high.school, illiterate, professional.course, university.degree, unknown*)  
5) **default** - есть ли дефолт по кредиту (Значения: *no, yes, unknown*)  
6) **housing** - есть ли кредит на жильё (Значения: *no, yes, unknown*)  
7) **loan** - есть ли потребительский кредит (Значения: *no, yes, unknown*)  
8) **contact** - способ связи c потенциальным клиентом (Значения: *cellular, telephone*)  
9) **month** - номер месяца, когда был крайний контакт с клиентом (Значения: *jan, feb, mar, ... , nov, dec*)  
10) **day_of_week** - день недели, когда был крайний контакт с клиентом (Значения: *mon, tue, wed, thu, fri*)  
11) **duration** - продолжительность крайнего звонка клиенту (Имеется ввиду, звонок, результат которого мы прогнозируем, в секундах)  
12) **campaign** - количество контактов с данным клиентом в течение текущей компании  
13) **pdays** - количество дней, прошедшее с предыдущего контакта (999 для новых клиентов)  
14) **previous** - количество контактов с данным клиенто до текущей компании  
15) **poutcome** - результат предыдущей маркетинговой компании (Значения: *failure, nonexistent, success*)  
16) **emp.var.rate** - коэффициент изменения занятости - квартальный показатель  
17) **cons.price.idx** - индекс потребительских цен - месячный показатель  
18) **cons.conf.idx** - индекс доверия потребителей - ежемесячный показатель  
19) **euribor3m** - euribor 3-месячный курс - дневной индикатор  
20) **nr.employed** - количество сотрудников - квартальный показатель  

Целевой признак:

21) **y** - подписал ли клиент срочный вклад (Значения: *yes, no*)

## 2-3. Первичный анализ данных и первичный визуальный анализ данных

Загрузим необходимые библиотеки:

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split, StratifiedKFold
from imblearn.over_sampling import RandomOverSampler
from sklearn import preprocessing
from sklearn.model_selection import validation_curve
from sklearn.svm import SVC
import time
from sklearn.linear_model import SGDClassifier
from xgboost import XGBClassifier
import warnings
from sklearn.tree import DecisionTreeClassifier
warnings.filterwarnings('ignore')

Загрузим данные из файла и зададим параметры вывода датафрейма, чтобы все колонки было видно.

In [ ]:
raw_data = pd.read_csv('bank-additional-full.csv', header = 0, sep = ';')
data = raw_data.copy()
pd.set_option('display.max_columns', 100)

Посмотрим на размер загруженного датасета:

In [ ]:
data.shape

Посмотрим, на содержимое датафрейма:

In [ ]:
data.head(5)

Проверим, есть ли в данных отсутствующие значения:

In [ ]:
data.isnull().sum()

Как видим, данные полные, пропусков нет, поэтому нет необходимости заполнять пропуски.  
Далее проверим данные на выбросы. Для начала сравним минимальное и максимальное значение со средним (для численных признаков):

In [ ]:
data.describe(include = ['int64', 'float64'])

В целом, по этим данным, нельзя сказать, что в данных есть откровенные выбросы. Однанако, такой проверки недостаточно,  
желательно ещё посмотреть графики зависимости целевого признака от каждого из признако, но это мы сделаем чуть позже,  
когда будем визуализировать признаки и зависимости.  

Далее посмотрим на распределения числовых признаков. Для этого воспользуемся библиотекой Seaborn.

In [ ]:
categorial = []
numerical = []
for feature in data.columns:
    if data[feature].dtype == object:
        categorial.append(feature)
    else:
        numerical.append(feature)

data[numerical].hist(figsize=(20,12), bins=100, color='lightgreen')

Из гистограмм мы видим, что **для каждого численного признака есть одно или несколько доминирующих отрезков значений**,  
из-за чего мы получили ярко выраженные пики.  

Далее осмотрим на категориальные признаки:

In [ ]:
data.describe(include = ['object'])

In [ ]:
plt.rcParams['axes.labelsize'] = 20
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['font.size'] = 20

fig, axes = plt.subplots(ncols=4, nrows = 3, figsize=(24, 18))
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.4)

for i in range(len(categorial)):
    data[categorial[i]].value_counts(normalize=True).plot(kind='bar', label=categorial[i], ax=axes[i//4, i%4], color='lightgreen')
    axes[i//4, i%4].set_title(categorial[i])
plt.tight_layout()

Как видим, для многих признаков, какая-то из групп привалирует, например, в датасете более половины клиентов женаты/замужем. 

Помимо этого мы видим, что целевой признак несбалансирован. **Количество положительных исходов существенно ниже, чем отрицательных**, что вполне естественно для телефонного маркетинга. Вследствие этого, возникает проблема с тем, что многие методы чувствительны к несбалансированности признаков. Данную проблему мы дальше постораемся решить.

Посмотрим на матрицу корреляции (для числовых признаков):

In [ ]:
correlation_table = data.corr()
correlation_table

Визуализируем матрицу корреляции:

In [ ]:
sns.heatmap(correlation_table)

Как видно из таблицы и тепловой карты **euribor3m и nr.employed сильно коррелируют с emp.var.rate**, впоследствии, на этапе отбора признаков мы их удалим, когда будем отбирать признаки.

Целевой признак показывает положительный ли результат телефонного звонка в ходе маркетинговой компании.  
Обозначим положительный исход как 1, отрицательный как 0:

In [ ]:
data['y'] = data['y'].map({'yes': 1, 'no': 0})

Посмотрим на визуализированные зависимости вещественных признаков от целевого признака:

In [ ]:
fig, axes = plt.subplots(ncols=4, nrows = 3, figsize=(24, 18))
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.4)

for i in range(len(numerical)):
    data.plot(x=numerical[i], y = 'y', label=numerical[i], ax=axes[i//4, i%4], kind='scatter', color='green')
    axes[i//4, i%4].set_title(numerical[i])
plt.tight_layout()

Как видим, есть точки, которые можно интерпретировать как выбросы есть, однако, я бы не спешил их удалять, т.к. мне они не кажутся выбросами в чистом виде. Т.е. это врядли ошибки и выбросы слишком сильные, пока я их оставлю. К тому же, для начала я намерен использовать RandomForest, а деревья, как известно, устойчивы к выбросам.

Категориальные признаки необходимо сделать числовыми, для этого я использую LabelEncoder. OneHotEncoder не использую, т.к. для категориальных признаков в дальнейшем буду считать WOE (см. дальше), поэтому нам неважно как занумеровывать категории.

In [ ]:
categorial.remove('y')
data[categorial] = data[categorial].apply(LabelEncoder().fit_transform)

In [ ]:
fig, axes = plt.subplots(ncols=4, nrows = 3, figsize=(24, 18))
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.4)

for i in range(len(categorial)):
    sns.countplot(x=categorial[i], hue='y', data=data, ax=axes[i//4, i%4])
    axes[i//4, i%4].set_title(categorial[i])

plt.tight_layout()

Из данных графиков видно некоторые зависимости, но лучше посмотреть в несколько другом виде.  
Визуализируем **долю положительных откликов** по группам:

In [ ]:
fig, axes = plt.subplots(ncols=4, nrows = 3, figsize=(24, 18))
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.4)

for i in range(len(categorial)):
    data.groupby(categorial[i])['y'].mean().plot(kind='bar', ax=axes[i//4, i%4], color='green')
    axes[i//4, i%4].set_title(categorial[i])

plt.tight_layout()

В таком виде, графики уже более интересные. Дак мы видим, для многих признаков, шанс положительного отклика существенно выше.  
Как я уже говорил, я панирую использовать WOE для работы с категориальными признаками.  
Помимо этого, исходя из данных графиков, напрашивается идея сделать более укрупненные категории. Например для образования объединить категории следующим образом:  
1) 0, 3, 5  
2) 1, 2  
3) 6, 7  
4) 4  
Однако, прелесть WOE в том, что как раз для значений признака, для которых доля положительных откликов сходная, будут близки и значения WOE. Поэтому, из графиков понятно, что применение WOE в данной задаче обосновано и желательно. 

Также мы видим, что признаки **housing**, **loan** и **day_of_week** врядли нам чем-то помогут, т.к. судя по графикам, доля положительных откликов от них практически не зависит. *Осторожно, спойлеры.* Далее мы их удалим, но перед этим посмотрим на них с точки зрения Information Value и увидим, что они действительно неинформативны.

## 4. Инсайты   
   
Пропусков в данных у нас нет, явных выбросов, которые стоило бы вырезать, на мой взгляд тоже нет. 
   
**euribor3m** и **nr.employed** сильно коррелируют с **emp.var.rate**.   
Напомню, **emp.var.rate** - коэффициент изменения занятости - квартальный показатель, **euribor3m** - euribor 3-месячный курс - дневной индикатор, **nr.employed** - количество сотрудников - квартальный показатель. Корреляция изменения занятости и собственно количества занятых вопросов не вызывает, а вот корреляция с Euribor (Euro Interbank Offered Rate, европейская межбанковская ставка предложения) - это интересно. Межбанковская ставка - это один из основных макропараметров экономики государства, т.е. влияет на темпы роста экономики, которые естественным образом влияют на занятость. Логично, но такая сильная корреляция подозрительна :). Слишком быстрый отклик рынка труда.

## 5. Выбор метрики

Цель банка состоит в том, чтобы снизить затраты на обзвон всех клиентов, сузив группу обзвона до той группы, для которой наиболее высока вероятность положительного отклика. Для банка важно снизить количество звонков не потеряв при этом клиентов, поэтому наиболее **критичными для банка являются ошибки FalseNegative**, т.к. банк теряет клиентов. Затраты на лишний звонок ниже, чем убыток от потерянного клиента. Исходя из этого, предпочитетльной метрикой будет **Recall**,  которая показывает, какую долю объектов положительного класса из всех объектов положительного класса нашел алгоритм. Однако, **Recall** метрика слишком "однобокая", т.к. стимулирует модель ставить всем объектам метки "1". Поэтому, более предпочтительным я считаю использование **ROC-AUC**, но при этом считаю важным следить отдельно за значением **Recall**. Ниже проиллюстрируем, почему **не стоит использовать Acuracy**

Что будет, если мы всем объектам поставим метки "0"?

In [ ]:
print ('Accuracy = ', round((1 - data['y'].mean())*100, 1))
print ('Recall = ', round(recall_score(data['y'], [0]*len(data['y'])), 1))
print ('ROC-AUC = ', round(roc_auc_score(data['y'], [0]*len(data['y'])), 1))

Таким образом, мы имеем точность модели 88.7%, однако банк имеет нулевую прибыль, т.к. обзвон потенциальных клиентов не производился.  
Основная причина такой ситуации в том, что целевые классы очень несбалансированны, положительных откликов всего 11.3%,  
поэтому мы предпочтём **ROC-AUC**

In [ ]:
data['y'].value_counts(normalize=True).apply(lambda x: round(x*100, 1))

А если мы всем объектам поставим метки "1"?

In [ ]:
print ('Accuracy = ', round((data['y'].mean())*100, 1))
print ('Recall = ', round(recall_score(data['y'], [1]*len(data['y'])), 1))
print ('ROC-AUC = ', round(roc_auc_score(data['y'], [1]*len(data['y'])), 1))

Как мы видим, recall максимальный, но при этом мы обзваниваем всех

## 6. Выбор модели

Как я говорил в начале своей работы, я разделю решение на 2 части:  
**1) "Банковское"**: логистическая регрессия + WOE. Выбор обоснован тем, что Логистическая Регрессия хорошо интерпретируема и используется в реальном банковском моделировании рисков. Однако, Логистическая Регрессия не работает с категориальными признаками, вернее сказать, что работает, но для этого значения категориальных признаков должны быть сравнимы. В том числе для этого вводится понятие WOE (Weight of evidence).   
**2) Свободное от ограничений решение**: в нем я хочу исследовать RandomForest и XGBoost. Делаю ставку на RandomForest, т.к. он устойчив к выбросам (которые я не удалял, т.к. не уверен, что они не важны) и т.к. многие признаки категориальные, значит деревья должны себя показать лучше на таких признаках. Данные модели будут тоже использоваться в сочетании с расчетом WOE (см. конец пункта 4). Также попробую XGBoost на деревьях в соответствии с той же логикой. И в итоге, попробую их объединить.

## 7. Предобработка данных

Удалим признаки **euribor3m** и **nr.employed**, которые сильно коррелируют с **emp.var.rate** (см. пункт 2-3)

In [ ]:
data.drop(['euribor3m', 'nr.employed'], axis=1, inplace=True)

Удалим признак **duration**, т.к. он показывает продолжительность разговора, необходимость которого мы хотим предсказать

In [ ]:
data.drop(['duration'], axis=1, inplace=True)

**Посчитаем WOE**:

![title](../../img/WOE.jpg)

Функция расчёта **Information Value**:

In [ ]:
def Information_Value(x, y):
    df = pd.DataFrame({'x': x, 'y': y})
    good = df.groupby('x')['y'].sum() / np.sum(df['y'])
    bad = (df.groupby('x')['y'].count() - df.groupby('x')['y'].sum()) / (len(df['y']) - np.sum(df['y']))
    WOE = np.log((good+0.000001) / bad)
    IV = (good - bad)*WOE
    return IV.sum()

Функция расчёта **WOE**:

In [ ]:
def WOE(x, y):
    df = pd.DataFrame({'x': x, 'y': y})
    good = df.groupby('x')['y'].sum() / np.sum(df['y'])
    bad = (df.groupby('x')['y'].count() - df.groupby('x')['y'].sum()) / (len(df['y']) - np.sum(df['y']))
    WOE = np.log((good+0.000001)/ bad)
    WOE = pd.Series(WOE).to_dict()
    return x.apply(lambda x: WOE.get(x))

WOE можно считать для значений признака, однако, учитывая формулу WOE, делать это для вещественных признаков не очень правильно. Лучше разбить диапазон значений на интервалы, закодировать их и посчитать WOE. Отсюда вытекает задача, как оптимально разбить значения признаков на интервалы. Для решения этой задачи, я решил для каждого признака обучать DecisionTreeClassifier и из него получать границы разбиение признака, при которых score максимальный.

In [ ]:
# Функция, которая возвращает границы разбиений
def get_bondaries(x_bondaries, y_bondaries):
    parameters = {'max_depth':[x for x in range(1, 21)], 'min_samples_leaf': [5, 10, 20, 30, 50, 70, 100, 150, 200, 300, 400, 500, 1000, 2000, 5000, 10000, 20000]}
    dtc = DecisionTreeClassifier(random_state=17)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)
    clf = GridSearchCV(dtc, parameters, scoring='roc_auc', cv=skf)
    clf.fit(pd.DataFrame(x_bondaries), y_bondaries)
    print('Best parameters for DT: ', clf.best_params_)
    print('ROC_AUC score: ', round(clf.best_score_, 4))
    tree = clf.best_estimator_
    tree.fit(pd.DataFrame(x_bondaries), y_bondaries)
    print('Boundaries: ', np.sort([x for x in tree.tree_.threshold if x!=-2]))
    return np.sort([x for x in tree.tree_.threshold if x!=-2])

Расчитываем границы разбиений для каждого признака. Посмотрим параметры дерева, которое делит признаки на интервалы, а так же на само разбиение.

In [ ]:
bondaries = dict()
for col in data.columns:
    if col != 'y':
        print (col)
        bondaries[col] = get_bondaries(data[col], data['y'])
        print ('--------------')

In [ ]:
# Функция каторая режет признак на интервалы и кодирует их
def splitter(x, col_bondaries):
    for i in range(len(col_bondaries)):
        if i>0:
            if x>col_bondaries[i-1] and x<=col_bondaries[i]:
                return i
        if i==0:
            if x<=col_bondaries[i]:
                return i
        if i==len(col_bondaries)-1:
            if x>col_bondaries[i]:
                return i+1

Считаем WOE для признаков:

In [ ]:
data_woe = pd.DataFrame()
for col in data.columns:
    if col != 'y' and col!='default' and col!='previous':
        data_woe[col] = data[col].apply(lambda x: splitter(x, bondaries[col]))
        data_woe[col] = WOE(data_woe[col], data['y'])
    if col=='default' or col=='previous':
        data_woe[col] = data[col]

Посмотрим на **Information Value** категориальных признаков:

In [ ]:
for feature in data_woe.columns:
    print('IV of ', feature, ' = ', Information_Value(data_woe[feature], data['y']), '\n')

Сравнивая Information value, видим, что признаки **housing**, **loan** и **day_of_week** являются неинформативными, поэтому мы их можем удалить. Кстати говоря, это же мы видели, когда строили графики, показывающие долю положительных откликов в зависимости от значения признака.

In [ ]:
data_woe.drop(['housing', 'loan', 'day_of_week'], axis=1, inplace=True)

## 8. Кросс-валидация и настройка гиперпараметров

Отложим выборку для теста:

In [ ]:
df, df_test, y, y_test = train_test_split(data_woe, data['y'], test_size=0.3, stratify=data['y'], random_state=17)

## 8.1 Логистическая регрессия

Подберем параметры с помощью **GridSearchCV**:

In [ ]:
parameters = {'penalty':['l1', 'l2'], 'C':[0.12, 0.11, 0.1, 0.09, 0.08, 0.07, 0.06, 0.05, 0.04, 0.03, 0.02, 0.01], 'class_weight':[None, 'balanced']}
LR = LogisticRegression(random_state=17, n_jobs=-1)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)
clf_lr = GridSearchCV(LR, parameters, scoring='roc_auc', cv=skf)
clf_lr.fit(data_woe, data['y'])
print('Best parameters: ', clf_lr.best_params_)
print('ROC_AUC score: ', round(clf_lr.best_score_, 4))

Теперь посмотрим качество на **отложенной выборке** и заодно оценим время работы **Logit** с наилучшими параметрами:

In [ ]:
start = time.time()
LR_control = LogisticRegression(C=0.04, penalty='l1')
LR_control.fit(df, y)
print('ROC_AUC on Control data: ', round(roc_auc_score(y_test, LR_control.predict_proba(df_test)[:,1]), 4))
print('Overall time: ', round(time.time()-start, 1))

## 8.2 Случайный лес

Теперь попробуем **RandomForest**:

In [ ]:
parameters = {'n_estimators':[280, 290, 300, 310, 320, 330], 'max_depth':[8, 9, 10, 11, 12, 15], 'min_samples_leaf': [1, 10, 30, 50, 100]}
RFC = RandomForestClassifier(random_state=17, n_jobs=-1)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)
clf_rfc = GridSearchCV(RFC, parameters, scoring='roc_auc', cv=skf)
clf_rfc.fit(data_woe, data['y'])
print('Best parameters: ', clf_rfc.best_params_)
print('ROC_AUC score: ', round(clf_rfc.best_score_, 4))

Результат на **отложенной выборке** с наилучшими параметрами:

In [ ]:
start = time.time()
RF_control = RandomForestClassifier(max_depth=9, n_estimators=290)
RF_control.fit(df, y)
print('ROC_AUC on Control data: ', round(roc_auc_score(y_test, RF_control.predict_proba(df_test)[:,1]), 4))
print("Overall time:", round(time.time()-start, 1))

Как мы видим, на отложенной выборке себя показывает **RandomForest** с **максимальной глубиной = 9** и **Количеством деревьев = 290**. Для сравнения, посмотрим результат работы **XGBoost**:

## 8.3 XGBoost

In [ ]:
parameters = {'n_estimators':[40, 50, 60, 100, 200, 300], 'max_depth':[2, 3, 4, 5, 6]}
xgb = XGBClassifier(random_state=17, n_jobs=-1)
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=17)
clf = GridSearchCV(xgb, parameters, scoring='roc_auc', cv=skf)
clf.fit(data_woe, data['y'])
print('Best parameters: ', clf.best_params_)
print('ROC_AUC score: ', round(clf.best_score_, 4))

In [ ]:
start = time.time()
xgb_control = XGBClassifier(max_depth=4, n_estimators=60, random_state=17, n_jobs=-1)
xgb_control.fit(df, y)
print('ROC_AUC on Control data: ', round(roc_auc_score(y_test, xgb_control.predict_proba(df_test)[:,1]), 4))
print("Overall time:", round(time.time()-start, 1))

Как видим, все три алгоритма показали близкие результаты. **XGBoost** и **Random Forest** чуть впереди, но **Логит** показал достойные результаты.

## 9. Создание новых признаков

В случае данного датасета, большинство признаков, которые логично создать, не имеет смысла вводить в модель, т.к. зависимости, которые они иллюстрируют должны хорошо выделяться случайным лесом и без создания новых признаков. Однако, есть признак, который хочется добавить в модель. У нас есть признак **pdays** (количество дней, прошедшее с предыдущего контакта) и **previous** (количество контактов с данным клиенто до текущей компании). В данном случае, количество контактов с клиентом разделим на количество дней с последнего контакта. Логика в том, что чем больше времени прошло, тем менее важно, как часто ранее общались с клиентом. Назовем это признак **previous_with_memory**.

In [ ]:
def previous_with_memory(x):
    if x[0] == 0:
        return 0
    else:
        return x[1]/x[0]

In [ ]:
data_new = data_woe.copy()
data_new['previous_with_memory'] = raw_data[['pdays', 'previous']].apply(previous_with_memory, axis=1)

In [ ]:
df, df_test, y, y_test = train_test_split(data_new, data['y'], test_size=0.3, stratify=data['y'], random_state=17)

In [ ]:
start = time.time()
RF_control = RandomForestClassifier(max_depth=9, n_estimators=290, random_state=17, n_jobs=-1)
RF_control.fit(df, y)
print('ROC_AUC on Control data: ', round(roc_auc_score(y_test, RF_control.predict_proba(df_test)[:,1]), 4))
print("Overall time:", round(time.time()-start, 1))


Для сравнения, запустим **XGBoost** на новых признаках:

In [ ]:
start = time.time()
xgb_control = XGBClassifier(max_depth=4, n_estimators=60, random_state=17, n_jobs=-1)
xgb_control.fit(df, y)
print('ROC_AUC on Control data: ', round(roc_auc_score(y_test, RF_control.predict_proba(df_test)[:,1]), 4))
print("Overall time:", round(time.time()-start, 1))

Как мы видим, прироста мы не увидели, поэтому не будем его использовать. Помимо данного признака, вариантов для создания признаков я не вижу.

##  Ансамбль  
Попробуем объединить алгоритмы **Random Forest** и **XGBoost** в ансамбль

In [ ]:
from sklearn.ensemble import VotingClassifier
clf1 = LogisticRegression(C=0.04, penalty='l1', random_state=17, n_jobs=-1)
clf2 = RandomForestClassifier(max_depth=9, n_estimators=290, random_state=17, n_jobs=-1)
clf3 = XGBClassifier(max_depth=4, n_estimators=60, random_state=17, n_jobs=-1)

eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('xgb', clf3)], voting='soft')
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

for clf, label in zip([clf2, clf3, eclf], ['Random Forest', 'XGBoost', 'Ensemble']):
    scores = cross_val_score(clf, data_woe, data['y'], cv=skf, scoring='roc_auc', n_jobs=-1)
    print("ROC-AUC: %0.4f (+/- %0.4f) [%s]" % (scores.mean(), scores.std(), label))

Объединение не дало прирост качества и незначительно увеличило устойчивость алгоритма.

## 10. Кривые валидации и обучения 
Кривые буду строить для **Random Forest**, т.к. сделал на него основную ставку. В целом он незначительно уступил XGBoost

In [ ]:
plt.figure(figsize=(8, 8))
param_range = [2, 5, 10, 50, 100, 150, 200, 250, 300, 350, 400]
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)
train_scores, test_scores = validation_curve(
    RandomForestClassifier(max_depth=9, random_state=17), data_woe, data['y'], param_name='n_estimators', param_range=param_range,
    cv=skf, scoring="roc_auc", n_jobs=-1)

train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

plt.title("Validation Curve with Random Forest")
plt.xlabel("n_estimators")
plt.ylabel("Score")
plt.ylim(0.77, 0.84)
lw = 2
plt.semilogx(param_range, train_scores_mean, label="Training score",
             color="darkorange", lw=lw)
plt.fill_between(param_range, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.2,
                 color="darkorange", lw=lw)
plt.semilogx(param_range, test_scores_mean, label="Cross-validation score",
             color="navy", lw=lw)
plt.fill_between(param_range, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.2,
                 color="navy", lw=lw)
plt.legend(loc="best")
plt.grid(color='grey', linestyle='-', linewidth=0.2)
plt.show()

In [ ]:
from sklearn.model_selection import learning_curve
plt.figure(figsize=(8, 8))

plt.title('Learning Curves for Random Forest')
plt.xlabel("Training examples")
plt.ylabel("Score")
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)
train_sizes, train_scores, test_scores = learning_curve(RandomForestClassifier(max_depth=9, n_estimators=290, random_state=17), data_woe, data['y'], cv=skf, n_jobs=1, train_sizes=np.linspace(.1, 1.0, 10), scoring='roc_auc')
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)
plt.grid()

plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color="r")
plt.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=0.1, color="g")
plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Cross-validation score")
plt.legend(loc="best")
plt.show()

Судя по кривой обучения, в целом, данных достаточно, но, видимо, некоторую прибавку качества за счёт увеличения датасета всё же можно было получить.  
Из кривой валидации видно, что основной рост качества происходит до 100 деревьев, далее качество меняется незначительно как в плюс так и в минус.

## Итог

Алгоритмы основанные на ансамблях деревьев показали себя очень хорошо, что не удивительно, учитывая, что мы считали WOE (т.е. создали разбиения, получив категориальные признаки для которых на значениях задана операция сравнения), данные не разреженные, размер датасета не маленький.  
При этом логистическая регрессия хоть и уступает, но показывает себя достаточно неплохо, т.е. при серьезной работе с ней, можно получать неплохие результаты, при этом интерпретируемые, что является большим плюсом для некоторых бизнес-задач.

Олег Акимов  
Slack: @Oleg  
LinkedIn: www.linkedin.com/in/oleg-akimov-715532102/ 